## Advent of code 2019 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 8
# start_ts=1663483180
# mv ~/Downloads/input* data_src/2019-day-8-input.txt
# big input file looks like: big map on one line of [012] (100 layers of 150 chars)
# idea: part 1 chop single line into pieces of 25x6, then use collections.counter and sort
# part 2: break up into segs, do layering, then show result

sample1='''
123456789012
'''

sample2='''
0222112222120000
'''

sample1=open('data_src/2019-day-8-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
line=lines[0]
wid=25
hi=6
sz=wid*hi
segs=[]
for i in range(0, len(line), sz):
    seg=line[i:i+sz]
    segs.append(seg)
part1res=[ collections.Counter(list(seg)) for seg in segs ]
part1res=[ (cnt['0'], cnt['1']*cnt['2']) for cnt in part1res ]
part1res.sort(key=lambda tup: tup[0])
print(f'{part1res[0]=}')
print()
lay=''
for i in range(sz):
    for seg in segs:
        if seg[i] in {'0', '1'}:
            lay+=seg[i]
            break
    if len(lay)<i+1:
        lay+='2'
assert len(lay)==sz
for i in range(0, len(lay), wid):
    line=lay[i:i+wid]
    line=line.replace('0', ' ')
    print(line)

# part 1: 1935
# part 2: CFLUL


In [ ]:
# 2019 day 7
# start_ts=1662877877
# mv ~/Downloads/input* data_src/2019-day-7-input.txt
# big input file looks like: line of numbers up to 1002, intcode
# idea: part 1 parse ..., then ...

sample1='''
3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
'''

sample2='''
3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
'''

sample3='''
3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5
''' # part 2

sample4='''
3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10
'''

def fetch_param(data, opcode, i, offset):
    mode=0
    if offset==1:
        mode=(opcode//100) % 10
    elif offset==2:
        mode=(opcode//1000) % 10
    elif offset==3:
        mode=(opcode//10000) % 10
    else:
        print(f'invalid offset {offset}')
        assert False
    if mode==0: # position mode
        param=data[i+offset]
        return data[param]
    elif mode==1: # immediate mode
        param=data[i+offset]
        return param
    else:
        print(f'invalid mode {mode}')
        assert False

def run_opcodes(data, input=[], i_ptr=0):
    i=i_ptr
    inp_index=0
    out_res=None
    halted=False
    while True:
        if data[i]%100==1: # add
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=a+b
            i+=4
        elif data[i]%100==2: # mult
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=a*b
            i+=4
        elif data[i]%100==3: # input
            a=input[inp_index]
            inp_index+=1
            data[data[i+1]]=a
            i+=2
        elif data[i]%100==4: # output
            a=fetch_param(data, data[i], i, 1)
            #print(f'value "at" {data[i+1]} is {a}')
            out_res=a
            i+=2
            break # stop after every output!
        elif data[i]%100==5: # jump-if-true
            a=fetch_param(data, data[i], i, 1)
            if a!=0:
                i=fetch_param(data, data[i], i, 2)
            else:
                i+=3
        elif data[i]%100==6: # jump-if-false
            a=fetch_param(data, data[i], i, 1)
            if a==0:
                i=fetch_param(data, data[i], i, 2)
            else:
                i+=3
        elif data[i]%100==7: # less-than
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=1 if a<b else 0
            i+=4
        elif data[i]%100==8: # equals
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=1 if a==b else 0
            i+=4
        elif data[i]%100==99: # stop
            halted=True
            break
        else:
            print(f'unknown instruction {data[i]} at position {i}')
            assert False
    return (out_res, halted, i)

sample1=open('data_src/2019-day-7-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
bestres=None
bestphases=None
for phases in itertools.permutations(range(5, 10), 5):
    amps=[]
    for phase in phases:
        amps.append(dict(code=list(data0), pha=phase, firstrun=True, halted=False, i_ptr=0))
    ampi=0
    res=0
    res_E=None
    while True:
        amp=amps[ampi]
        if amp['halted']:
            break
        inp=[amp['pha'], res] if amp['firstrun'] else [res,]
        amp['firstrun']=False
        res,halted,i_ptr=run_opcodes(amp['code'], input=inp, i_ptr=amp['i_ptr'])
        amp['halted']=halted
        amp['i_ptr']=i_ptr
        if res is not None and ampi==4:
            res_E=res
        ampi=(ampi+1)%5
    if bestres is None or res_E>bestres:
        bestres=res_E
        bestphases=''.join([str(i) for i in phases])
print(bestres, bestphases)

# part 1: 21860 04213
# part 2: 2645740 65789 (python 3.10.4 - ca. 0.6s, pypy 3.9 ca. 0.2s)

In [ ]:
# 2019 day 6
# start_ts=1662702691
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: ca. 1500 lines of orbits
# idea: part 1 parse lines, maintain map of sets of direct children, depth first count

sample1='''
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
'''

sample2='''
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
'''

def count_paths(children, node, depth):
    count=0
    cs=children.get(node)
    if cs:
        for child in cs:
            count+=count_paths(children, child, depth+1)
    return depth+count

def root_path(parents, node):
    path=[node]
    while node := parents.get(node):
        path.insert(0, node)
    return path

def remove_common(p1, p2):
    p1=list(p1)
    p2=list(p2)
    while len(p1)>0 and len(p2)>0 and p1[0]==p2[0]:
        p1.pop(0)
        p2.pop(0)
    return p1, p2

sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split(')') for s in lines ]
children={}
parents={}
for a,b in data:
    cs=children.setdefault(a, set())
    cs.add(b)
    parents[b]=a
print('part 1: ', count_paths(children, 'COM', 0))
youp=root_path(parents, 'YOU')
sanp=root_path(parents, 'SAN')
#print(youp, sanp)
youp, sanp=remove_common(youp, sanp)
print('part 2: ', len(youp)+len(sanp)-2)

# part 1: 273985
# part 2: 460

In [ ]:
# 2019 day 5
# start_ts=1662270754
# mv ~/Downloads/input* data_src/2019-day-5-input.txt
# big input file looks like: single line of numbers up to 99999
# idea: part 1 parse ..., then ...

sample1='''
3,0,4,0,99
'''

sample2='''
1002,4,3,4,33,4,4,99
'''

sample3='''
3,9,8,9,10,9,4,9,99,-1,8
'''

sample4='''
3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
'''

def fetch_param(data, opcode, i, offset):
    mode=0
    if offset==1:
        mode=(opcode//100) % 10
    elif offset==2:
        mode=(opcode//1000) % 10
    elif offset==3:
        mode=(opcode//10000) % 10
    else:
        print(f'invalid offset {offset}')
        assert False
    if mode==0: # position mode
        param=data[i+offset]
        return data[param]
    elif mode==1: # immediate mode
        param=data[i+offset]
        return param
    else:
        print(f'invalid mode {mode}')
        assert False

def run_opcodes(data, input=1):
    i=0
    while True:
        if data[i]%100==1: # add
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=a+b
            i+=4
        elif data[i]%100==2: # mult
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=a*b
            i+=4
        elif data[i]%100==3: # input
            a=input
            data[data[i+1]]=a
            i+=2
        elif data[i]%100==4: # output
            a=fetch_param(data, data[i], i, 1)
            print(f'value "at" {data[i+1]} is {a}')
            i+=2
        elif data[i]%100==5: # jump-if-true
            a=fetch_param(data, data[i], i, 1)
            if a!=0:
                i=fetch_param(data, data[i], i, 2)
            else:
                i+=3
        elif data[i]%100==6: # jump-if-false
            a=fetch_param(data, data[i], i, 1)
            if a==0:
                i=fetch_param(data, data[i], i, 2)
            else:
                i+=3
        elif data[i]%100==7: # less-than
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=1 if a<b else 0
            i+=4
        elif data[i]%100==8: # equals
            a=fetch_param(data, data[i], i, 1)
            b=fetch_param(data, data[i], i, 2)
            data[data[i+3]]=1 if a==b else 0
            i+=4
        elif data[i]%100==99: # stop
            break
        else:
            print(f'unknown instruction {data[i]} at position {i}')
            assert False

sample1=open('data_src/2019-day-5-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ]
run_opcodes(data0, input=5)
#exit()

# part 1: 9219874
# part 2: 5893654

In [ ]:
# 2019 day 4
# start_ts=1662184709
# mv ~/Downloads/input* data_src/2019-day-4-input.txt (N/A)
# big input file looks like: see input1 below, just a range
# idea: part 1 parse range, then iterate over all options as list of ints, based on last rule
#  part 2: small list, so just save all built matching sequences from part 1 and filter on subsequence length
#  (does the built sequence has a subsequence of repeating digits of 2?)

input1='137683-596253'

sample1='''
111111-111112
''' # should be 2

sample2='''
223450-223450
''' # should be 0

sample3='''
11-12
'''

def in_range(build, minnum, maxnum):
    return minnum<=int(build)<=maxnum

def do_count(n, build, minnum, maxnum, hasdouble, res):
    '''gather in res all strings of n digits, non-decreasing, with at least a double (two adjacent same digits)'''
    if len(build)==n:
        if hasdouble and in_range(build, minnum, maxnum):
            res.append(build)
        return
    if len(build)>0:
        lastc=build[-1]
    else:
        lastc='0'
    for ci in range(int(lastc), 10):
        newc=str(ci)
        do_count(n, build+newc, minnum, maxnum, hasdouble or (len(build)>0 and newc==lastc), res)

def do_count_seq2(build):
    '''count lengths of strings of repeated digits, put in set, return True if there's a two'''
    lastc=None
    n=0
    seqs=set()
    for c in list(build):
        if c==lastc:
            n+=1
        else:
            if n>0:
                seqs.add(n)
            n=1
        lastc=c
    if n>0:
        seqs.add(n)
    return 2 in seqs

#sample1=open('data_src/2019-day-4-input.txt').read()
lines=[s for s in input1.splitlines() if len(s)>0 ]
data=[ str(s) for s in lines[0].split('-') ]
#data2=[ [ int(s) for s in list(d)] for d in data ]
n=len(data[0])
assert len(data[1])==n
res=[]
do_count(n, '', int(data[0]), int(data[1]), False, res)
print(len(res))
cnt=0
for build in res:
    if do_count_seq2(build):
        cnt+=1
print(cnt)
# part 1: 1864
# part 2: 1258

In [ ]:
# 2019 day 3
# start_ts=1661581356
# mv ~/Downloads/input* data_src/2019-day-3-input.txt
# big input file looks like: list of codes on two lines
# idea: part 1 parse and split each line, draw into grid, count lines per cell, then list crossoverpoints and take closest

sample1='''
R8,U5,L5,D3
U7,R6,D4,L4
'''

sample2='''
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83
'''

sample3='''
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7
'''

def update_cell(grid, x, y, lineno, steps):
    cell=grid.setdefault( (x, y), {})
    if lineno in cell:
        return 
    cell[lineno]=steps

def draw_lines(grid, lines, lineno):
    x=0
    y=0
    steps=0
    for line in lines:
        direction=line[0]
        distance=int(line[1:])
        for i in range(distance):
            if direction=='U':
                y-=1
            elif direction=='D':
                y+=1
            elif direction=='L':
                x-=1
            elif direction=='R':
                x+=1
            steps+=1
            update_cell(grid, x, y, lineno, steps)

def crossovers_dist(grid): # for part 1
    crossovers=[ k for k,v in grid.items() if len(v)>1 ]
    cr_dists=[ abs(tup[0])+abs(tup[1]) for tup in crossovers ]
    cr_dists.sort()
    return cr_dists

def steps_dist(grid): # for part 2
    crossovers=[ k for k,v in grid.items() if len(v)>1 ]
    cr_steps=[ grid[tup][0]+grid[tup][1] for tup in crossovers ]
    cr_steps.sort()
    return cr_steps

sample1=open('data_src/2019-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ lin.split(',') for lin in lines ]
assert len(data)==2
grid={} # maps x,y tuple to map of lineno to steps to reach
draw_lines(grid, data[0], 0)
draw_lines(grid, data[1], 1)
res=steps_dist(grid)
print(res[0])

# 27330

In [ ]:
# 2019 day 2
# start_ts=1643360000
# mv ~/Downloads/input data_src/2019-day-2-input.txt
# big input file looks like: about 100 numbers
# idea: part 1 parse as single list, then simply run

sample1='''
1,9,10,3,2,3,11,0,99,30,40,50
'''

sample2='''
1,1,1,4,99,5,6,0,99
'''

def run_opcodes(data):
    i=0
    while True:
        if data[i]==1:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a+b
            i+=4
        elif data[i]==2:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a*b
            i+=4
        elif data[i]==99:
            break
        else:
            assert False

sample1=open('data_src/2019-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ]
for noun in range(100):
    for verb in range(100):
        data=list(data0)
        data[1]=noun
        data[2]=verb
        run_opcodes(data)
        if data[0]==19690720:
            answer=100*noun+verb
            print(f'{answer=}')
            exit()

In [ ]:
# 2019 day 1
# start_ts=1661070296
# mv ~/Downloads/input* data_src/2019-day-1-input.txt
# big input file looks like: list of numbers
# idea: part 1 parse ..., then ...

sample1='''
100756
'''

def fuel(mass):
    res=0
    while mass>0:
        fuel=mass//3 - 2
        if fuel>=0:
            res+=fuel
            mass=fuel
        else:
            break
    return res

sample1=open('data_src/2019-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines ]
data=[ fuel(i) for i in data ]
sum(data) # should be 5003530

In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed